In [2]:
import pandas as pd
import numpy as np
import scipy.stats as S
import random

In [3]:
raw_data=pd.read_csv("data.csv")

In [4]:
raw_data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
data=raw_data.iloc[:,1:3]

In [6]:
print(data.head())

  diagnosis  radius_mean
0         M        17.99
1         M        20.57
2         M        19.69
3         M        11.42
4         M        20.29


In [7]:
b_data = data[data["diagnosis"] == 'B']

In [8]:
m_data = data[data["diagnosis"] == 'M']

In [9]:
print(b_data.shape)

(357, 2)


In [10]:
print(m_data.shape)

(212, 2)


In [12]:
mdatatraining=m_data.iloc[0:200,:]
print(mdatatraining.head())

  diagnosis  radius_mean
0         M        17.99
1         M        20.57
2         M        19.69
3         M        11.42
4         M        20.29


In [13]:
bdatatraining=b_data.iloc[0:200,:]
print(bdatatraining.head())

   diagnosis  radius_mean
19         B       13.540
20         B       13.080
21         B        9.504
37         B       13.030
46         B        8.196


In [14]:
pcapb = bdatatraining.shape[0]/400
print(pcapb)

0.5


In [15]:
pcapm = mdatatraining.shape[0]/400
print(pcapm)

0.5


In [16]:
mucapb = (bdatatraining['radius_mean']).mean() 
print(mucapb)

11.987175000000002


In [17]:
mucapm = (mdatatraining['radius_mean']).mean()
print(mucapm)

17.344850000000005


In [22]:
sigmacapb = (bdatatraining['radius_mean']).std()
print(sigmacapb)

1.7110478362291106


In [23]:
sigmacapm = (mdatatraining['radius_mean']).std()
print(sigmacapm)

3.189971562624307


In [24]:
bdatatesting =b_data.iloc[200:,:]
print(bdatatesting.head())

    diagnosis  radius_mean
361         B        13.30
362         B        12.76
363         B        16.50
364         B        13.40
367         B        12.21


In [25]:
mdatatesting =m_data.iloc[200:,:]
print(mdatatesting.head())

    diagnosis  radius_mean
516         M        18.31
517         M        19.89
521         M        24.63
533         M        20.47
535         M        20.55


In [26]:
def Univariatenaivebayesclassifier(oneexample):
    
    # going to calculate :
    # p(stest/ tumorisbenign) =normal distribution pdf with mean = mucapb and std=sigmacapb
    pstesttumorisbenign = S.norm.pdf(oneexample,mucapb,sigmacapb)
    
    # going to calculate: 
    # p(stest/tumor is benign) * p(tumor is benign)
    numerator = pstesttumorisbenign * pcapb
    
    #going to calculate:
    #p(stest/tumor is malignent) normal distribution pdf with mean = mucapm and std=sigmacapm
    pstesttumorismalignent =  S.norm.pdf(oneexample,mucapm,sigmacapm)
    
    #going to calculate:
    #p(stest/tumor is malignent) * p(tumor is malignent)
    secondpartofdenominator = pstesttumorismalignent * pcapm
    
    #bayes rule for p(tumor is benign/stest)
    ptumorisbenignstest = numerator/(numerator+secondpartofdenominator)
    
    return ptumorisbenignstest

In [27]:
truepositive = 0               # right answer and tumor is malignent
falsepositive = 0              # wrong answer and tumor is malignent
truenegative = 0               # right answer and tumor is benign
falsenegative = 0              # wrong answer and tumor is benign
for i in range(0,len(bdatatesting)):
    
    ptumorisbenignstest = Univariatenaivebayesclassifier(bdatatesting.iloc[i,1])
    
    if ptumorisbenignstest > 0.5:
        truenegative += 1
    else:
        falsepositive += 1

In [28]:
print(truenegative)

135


In [29]:
print(falsepositive)

22


In [30]:
for j in range(0,len(mdatatesting)):
    
    ptumorisbenignstest = Univariatenaivebayesclassifier(mdatatesting.iloc[j,1])
    
    if ptumorisbenignstest < 0.5:
        truepositive += 1
    else:
        falsenegative += 1

In [31]:
print(truepositive)

11


In [32]:
print(falsenegative)

1


In [35]:
precision = (truepositive + truenegative) / (mdatatesting.shape[0] + bdatatesting.shape[0]) * 100
print(precision)

86.3905325443787


In [36]:
recall= truepositive / mdatatesting.shape[0] * 100
print(recall)

91.66666666666666
